In [3]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

In [7]:
df = pd.read_csv("diabetes.csv")

In [8]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [60]:
X = df.drop("Outcome",axis = 1)

In [61]:
y = df["Outcome"]

In [65]:
X_tensor = torch.tensor(X.to_numpy(),dtype = torch.float32)
y_tensor = torch.tensor(y.to_numpy(),dtype = torch.float32)

In [66]:
X_tensor.dtype, y_tensor.dtype

(torch.float32, torch.float32)

In [67]:
data = TensorDataset(X_tensor,y_tensor) # Takes tensors

In [68]:
features, labels = data[0]

In [69]:
features

tensor([  6.0000, 148.0000,  72.0000,  35.0000,   0.0000,  33.6000,   0.6270,
         50.0000])

In [70]:
labels

tensor(1.)

In [82]:
loader = DataLoader(data,batch_size=4,shuffle = True)

In [83]:
loader

In [84]:
for batch_feature, batch_label in loader:
    # print(i) # 2-D tensor of features , 1-D tesnsor of lables with shape (2,) because batch size is 2.
    print("batch_feature ",batch_feature)
    print("batch_label ",batch_label)
    print("\n")

batch_feature  tensor([[  2.0000,  93.0000,  64.0000,  32.0000, 160.0000,  38.0000,   0.6740,
          23.0000],
        [  3.0000, 116.0000,   0.0000,   0.0000,   0.0000,  23.5000,   0.1870,
          23.0000],
        [  1.0000, 167.0000,  74.0000,  17.0000, 144.0000,  23.4000,   0.4470,
          33.0000],
        [  1.0000, 111.0000,  94.0000,   0.0000,   0.0000,  32.8000,   0.2650,
          45.0000]])
batch_label  tensor([1., 0., 1., 0.])


batch_feature  tensor([[9.0000e+00, 1.2400e+02, 7.0000e+01, 3.3000e+01, 4.0200e+02, 3.5400e+01,
         2.8200e-01, 3.4000e+01],
        [1.0000e+00, 1.2400e+02, 6.0000e+01, 3.2000e+01, 0.0000e+00, 3.5800e+01,
         5.1400e-01, 2.1000e+01],
        [2.0000e+00, 1.7500e+02, 8.8000e+01, 0.0000e+00, 0.0000e+00, 2.2900e+01,
         3.2600e-01, 2.2000e+01],
        [1.3000e+01, 1.0600e+02, 7.0000e+01, 0.0000e+00, 0.0000e+00, 3.4200e+01,
         2.5100e-01, 5.2000e+01]])
batch_label  tensor([0., 0., 0., 0.])


batch_feature  tensor([[  2.0000

In [184]:
model = nn.Sequential(
    nn.Linear(8,7),
    nn.Linear(7,5),
    nn.LeakyReLU(),
    nn.Linear(5,1) # for each row will have one score and sigmoid will bring in range of 0 and 1
    #Input: (batch_size, 8)
    # Output: (batch_size, 1),
)

In [185]:
oz = torch.optim.SGD(model.parameters(),lr = 0.0001)

In [186]:
loss_object = nn.BCEWithLogitsLoss() #BCEWithLogitsLoss: 1 output neuron + Sigmoid internally

In [187]:
loss_list = list()
for epoch in range(10):
 for i in loader:
    oz.zero_grad()
    features, labels = i # labels have shape (4,)
    # print(features) # features of shape (4,8)
    y_pred = model(features)
    # print(y_pred) # y_pred are of shape (4,1)
    loss = loss_object(y_pred,labels.reshape(-1,1)) # size (4,) to size (4,1) because it makes easy for loss function to calculate loss value element wise
    # shape of input = shape of output/ predictions
    loss.backward()
    oz.step()
 loss_list.append(loss)

In [188]:
loss_list

[tensor(0.6661, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.5631, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.5665, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.6163, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.9115, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.5758, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.6513, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.4297, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.6480, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 tensor(0.6080, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)]